In [1]:
import numpy as np
import math

def calc_from_genus(g, _a_, _b_, verbose=False):
    #Define things, collect info to build tableau, validate input
    while True:
        try:
            ########### this is new
            tab_dict = {}
            k = len(_a_)+2
        except ValueError:
            print("Oops! That doesn't look right... Try again!")
            continue

        #Remove boxes from each column (as a list)
        a = list(range(2,g+1)) #Symbols in column 1 of Lambda
        for n in _a_:
            a.remove(n) #Symbols in column 1 of lambda_D
        b = list(range(1,g)) #Symbols in column 0 of Lambda
        for n in _b_:
            b.remove(n) #Symbols in column 0 of lambda_D  

        #Verify that the given values form a tableau
        if all(a[n]>b[n] for n in range(g-k+1)) and len(a)==len(b):
            break
        else:
            print("Oops! This isn't a valid displacement tableau. Be sure the number of b-values smaller than each n is no more than the number of a values.")
            print(_a_)
            print(_b_)
            break
            #continue

    #Construct torsion profile
    def torsion(n):
        if a.count(n)==0 or b.count(n)==0: 
            return 0
        else:
            return b.index(n)-a.index(n)+1 #(y in column 0)-(x in column 1) +1 horizontal unit gives lattice distance

    _m_= [torsion(n) for n in range(2,g+1)]

    def calc_torsion(n): #To use in places where we do modular calculations since mod 0 breaks things
        if a.count(n)==0 or b.count(n)==0:
            return 104729
        else: 
            return torsion(n)


    #Build lambda_D
    lambda1 = Tableau([])
    for n in range(g-k+1):
        lambda1 = lambda1.add_entry((n,0),b[n])
        lambda1 = lambda1.add_entry((n,1),a[n])
    if verbose:
        print ("_m_=", _m_)
        print ("k=",k)
        print ("D")
        lambda1.pp()
    #Make a width function
    def width(t):
        return t.shape()[0] #takes the width of the top row



    #Calculate initial placement of each symbol
    def xi1(s): #for s in range(1,g+1) 
        if s<g+1 and b.count(s)>0:
            return b.index(s)
        elif s<g+1 and a.count(s)>0:
            return a.index(s)-1
        else:
            return -1


    #Calculate placement in subsequent tableaux
    def xi(c,s):
        if torsion(s)>0:
             return ((c*xi1(s))-((c-1)*(s-1))) % _m_[s-2]
        else:
             return ((c*xi1(s))-((c-1)*(s-1)))


    def tryj(i,j): #Build a tableau for the next value of c corresponding to a rank jump of j
        lambdac = Tableau([])
        coordinatesum = 0 #Tracks which diagonal we're on

        for coordinatesum in range(0,g): #range(0,(ranks[i-1]+heights[i-1])):
                x = max(0, (coordinatesum - (heights[i-1]-j-1))) #Returns 0 if coordinatesum is smaller than our max y value, 0 else
                y = coordinatesum - x

                while x < ranks[i-1]+k-j+1 and y>-1:
                    #Find entries above and left of (x,y)
                    if x == 0 and y == 0:
                        p = 0
                        q = 0
                    elif x == 0:
                        p = 0
                        q = lambdac.entry((y-1,0))
                    elif y == 0:
                        p = lambdac.entry((0,x-1))
                        q = 0
                    else: 
                        if ((y-1,x) in lambdac.cells()) and ((y,x-1) in lambdac.cells()):
                            p = lambdac.entry((y,x-1))
                            q = lambdac.entry((y-1,x))

                        else: #Break if we're missing a box, so the tableau can't be rectangular 
                            return lambdac

                    s = max(p,q)+1 #Smallest potential symbol to put in (x,y)

                    while ((y-x) - xi(i,s)) % calc_torsion(s) != 0: #Check xi^c_s
                        if s>g:

                            if lambdac.height() == 1: 
                                return lambdac
                            break
                        else:
                            s+=1

                    if s < g+1:
                        lambdac=lambdac.add_entry((y,x), s) #Add smallest possible symbol that has correct xi^c, if it exists
                        if s == g:
                            return lambdac 

                    if x==ranks[i-1]+k-j and y==heights[i-1]-j-1: #if we've built a rectangular tableau corresponding to a rank jump of j
                        return lambdac
                        #break
                    else:
                        x+=1
                        y-=1

                #coordinatesum += 1
        return lambdac


    ranks = [0,1] #We know the ranks of 0D and D; will add to this list as we build tableaux (but same as widths-1)
    heights = [0,g-k+1] #Record the size of lambda^c 



    def tabc(i): #Build the tableau corresponding to iD; must be run for all i>1, in order, since it's recursive
        #Try to make it j rows shorter and wider than lambda(i-1) for j starting at 1 and less than k, increase j until the result is rectangular
        j = 1 #=k-rank jump

        if tryj(i,j) == Tableau([]):
            return Tableau([])

        while tryj(i,j).is_rectangular() == False:  
            if j < k-1:
                j+=1

            else:
                return Tableau([])
        return tryj(i,j)

    #Calculate rank sequence and print tableau for each cD
    for c in range(2,(2*g-2)):
        thistableau = tabc(c)
        if thistableau == Tableau([]) or width(thistableau)-ranks[c-1]-1+heights[c-1]-thistableau.height()!=k:
            break
        else:
            heights.append(thistableau.height())
            ranks.append(width(thistableau)-1)
            if verbose:
                print (c, "D")
                thistableau.pp()
            ######## this is new
            tab_dict[c] = thistableau


    #add the rank of the first non-effective K-cD so we have a complete picture 
    ranks.append(len(ranks)*k-g)

    #Calculate Scrollar Invariants from rank sequence
    sis = {i : [] for i in range(0,k)}
    m = 0
    i = 0
    while i < k:
        if sis[i] == []:
            if all(ranks[c] >= i*c + i + c - m for c in range(0,len(ranks))):
                sis[i] = m
                i += 1
            else:
                m += 1

    #Calculate n
    maxmult = len(ranks)-1
    
    #Calculate ell 
    firstjump = 0
    while ranks[firstjump]==firstjump and firstjump < len(ranks)-1:
        firstjump += 1


    return {"ranks":ranks, "scrollar invariants":sis, "gonality":k, "torsion" :_m_, "n":maxmult, "ell":firstjump}

In [2]:
# only for generating ak_s

def to_remove(g, k, all_aks):
    for a1 in range(2, g):
        to_remove_recursive(a1, g, k, [], [], all_aks)

def to_remove_recursive(ai, g, k, aks, bks, all_aks):
    if len(aks) == 0:
        aks = [ai]
    if len(aks) == k-2:
        all_aks.append(aks)
        return
    index = len(aks)
    #print("ai:", ai, "end:", g-(k+2+index-1)+1)
    for anext in range(ai+1,g):
        to_remove_recursive(anext, g, k, aks + [anext], bks, all_aks)

    #print(aks)

In [3]:
#Run above code to generate data
aks =[]
genus = 10
to_remove(genus,3,aks)
dict_data = []
for i in range(0,len(aks)):
    for j in range(i,len(aks)):
        temp_dict = calc_from_genus(genus,aks[i],aks[j],verbose=False)
        if temp_dict == None:
            continue
        else:
            temp_dict["ak"] = aks[i][0]  #delete [0] for k>3
            temp_dict["bk"] = aks[j][0]
            temp_dict["genus"]  = genus
            dict_data.append(temp_dict)

In [ ]:
#Display data if you want
dict_data

In [4]:
#Write data to CSV file
import csv
csv_columns = ["genus","gonality","torsion","ranks","scrollar invariants","ak","bk","n","ell"]
csv_file = "g10k3.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")